# 문제 6

(kaggle 형 문제) prob4 를 바탕으로 아래 데이터셋을 만든다.

. prob6_train: DateHour 변수 기준으로 2021년 8월 14일 전(8월 14일 미포함) 데이터 (행의 수: 5256개)

. prob6_test: DateHour 변수 기준으로 2021년 8월 14일 이후(8월 14일 포함) 데이터 (행의 수: 744 개)

일 때, prob6_train으로 target을 예측하는 모델을 만들어, 

prob6_test의 target에 대한 MAE를 최소화하는 모델을 만든다. 

prob6_test의 예측 결과를 아래와 같은 형식으로 출력한다. 파일명은 answer6.csv 이다.

|DateHour|TotalHour|
|--------|---------|
|2021-08-14 00:00:00|102.607580|
|2021-08-14 01:00:00|94.078890|
....

# 시험장 환경 정보

Python: 3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]

|모듈|버젼|
|----|----|
|pandas|0.25.1|
|numpy|1.18.5|
|sklearn|0.21.3|
|scipy|1.5.2|
|mlxtend|0.15.0.0|
|statsmodels|0.11.1|

**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1


In [2]:
df_elec = pd.read_csv('elec.csv', parse_dates=['Date', 'DateHour'])
df_info = pd.read_csv('info.csv')
df_info['Date'] = pd.to_datetime(df_info['Date'])

df_elec1 = df_elec.pivot(
    index='DateHour', 
    columns='Minute', 
    values='Value').reset_index() # index에 위치한 DateHour를 컬럼에 위치시킵니다

holi = pd.to_datetime(["2021-01-01", "2021-02-11", "2021-02-12", "2021-03-01", "2021-05-05", "2021-05-19", "2021-08-16"]).date

# pd.Series.dt accessor를 통해 파생 변수들을 만듭니다.
df_elec1 = df_elec1.assign(
    DayName = df_elec1['DateHour'].dt.weekday, 
    Hour = df_elec1['DateHour'].dt.hour,
    AM = (df_elec1['DateHour'].dt.hour >= 12).astype('int'),
    Weekend_yn = df_elec1['DateHour'].dt.weekday.isin([5, 6]).astype('int'),
    Holiday_yn = df_elec1['DateHour'].dt.date.isin(holi).astype('int'), # df_elec1['DateHour'].dt.date는 pd.Series 입니다.
    Avg = df_elec1.mean(axis=1),
    TotalHour = df_elec1.sum(axis=1),
)

df_info1 = df_info.fillna(0)

df_basetable1 = df_elec1.merge(
    df_info1, 
    left_on='DateHour', 
    right_on='Date', how='inner'
).drop(columns='Date') # DateHour가 컬럼에 있으므로 DateHour, Date 모두 존재하여, Date는 삭제했습니다.
# shift를 이용합니다.

df_prob3 = df_basetable1.sort_values('DateHour')
df_prob3['target'] = df_prob3['TotalHour'].shift(-24)
df_prob4 = pd.concat(
    [df_prob3] + [
        df_prob3['TotalHour'].shift(24 * i).rename('lag_{}'.format(i)) for i in range(1, 7)
    ], axis=1
).dropna()

In [3]:
# shift를 사용하지 않고 target를 구하는 방법입니다. (값을 구하기만 하고, 따로 prob4로 만들지는 않았습니다.)
df_basetable1.join(
    df_basetable1[['DateHour','TotalHour']].assign(
        DateHour=lambda x: x['DateHour'] - pd.Timedelta(days=1)
    ).set_index('DateHour').rename(columns={'TotalHour': 'target'}),
    on='DateHour', how='inner'
)

,DateHour,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,...,TotalHour,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비,target
0,2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,...,245,0,-3.2,2.4,71,0.0,109.8,0.000000,1.5,253
1,2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,...,418,0,-4.5,1.5,77,0.0,109.8,0.000000,1.5,418
2,2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,...,415,0,-3.9,2.6,58,0.0,109.8,0.000000,1.5,415
3,2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,...,421,0,-4.1,2.6,56,0.0,109.8,0.000000,1.5,421
4,2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,...,427,0,-4.6,2.6,60,0.0,109.8,0.000000,1.5,427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6139,2021-09-13 19:00:00,162,160,148,122,0,19,1,0,0,...,592,2820,22.3,2.1,78,0.0,167.2,4.763514,1.5,613
6140,2021-09-13 20:00:00,113,122,122,126,0,20,1,0,0,...,483,14,22.2,1.3,78,0.0,167.2,0.028986,1.5,512
6141,2021-09-13 21:00:00,122,131,122,115,0,21,1,0,0,...,490,72,21.9,1.9,77,0.0,167.2,0.146939,1.5,513
6142,2021-09-13 22:00:00,97,108,122,113,0,22,1,0,0,...,440,11,21.7,1.4,77,0.0,167.2,0.025000,1.5,443


In [4]:
# 문제에서 제시한 데이터셋을 만듭니다.
df_prob6_train = df_prob4.loc[df_prob4['DateHour'] < pd.to_datetime('2021-08-14')]
df_prob6_test = df_prob4.loc[df_prob4['DateHour'] >= pd.to_datetime('2021-08-14')]
# 정답 target은 떼어냅니다.
s_kaggle_ans =  df_prob6_test.pop('target')

In [5]:
# Kaggle형 풀이 단계
# Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.
# Step 2: Baseline 모델을 만듭니다
# Step 3: 모델 선택 루틴을 만듭니다.
# Step 4: 모델 개선 작업을 합니다.

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# 시점을 기준으로 한 Hold-out 방법
# 문제에서 제시한 train/test 구분과 같이 검증셋도 시점 기준으로 검증train과 검증test(검증데이터)로 나눕니다.
df_train_eval = df_prob6_train.loc[df_prob6_train['DateHour'] < pd.to_datetime('2021-07-14')]
df_eval = df_prob6_train.loc[df_prob6_train['DateHour'] >= pd.to_datetime('2021-07-14')]
X_all = [i for i in df_train_eval.columns if i != 'target']
# 검증 루틴
def eval_model(model):
    model.fit(df_train_eval[X_all], df_train_eval['target'])
    return (
        mean_absolute_error(df_eval['target'], model.predict(df_eval[X_all])), 
        mean_absolute_error(df_train_eval['target'], model.predict(df_train_eval[X_all]))
    )

# 모델 선택 
def choose_model(model):
    # 전체 데이터셋으로 재학습
    model.fit(df_prob6_train[X_all], df_prob6_train['target'])
    # 평가 데이터 예측
    prd = model.predict(df_prob6_test[X_all])
    # 평가 결과 파일 제작
    pd.DataFrame({
        'DateHour': df_prob6_test['DateHour'],
        'TotalHour': prd
    }).to_csv('answer6.csv', index=None) # index=None, index는 파일에 미포함
    return prd

In [6]:
# Baseline 모델을 만듭니다.
from sklearn.svm import SVR
ct = ColumnTransformer([
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)])
])

reg_svm = make_pipeline(ct, SVR(C=10, kernel='rbf', gamma=0.1))
eval_model(reg_svm)

(148.65000821712516, 61.50534596142382)

In [7]:
# Baseline 모델을 제출합니다.
prd = choose_model(reg_svm)
# 자가 채점합니다.
mean_absolute_error(s_kaggle_ans, prd)

30.017658858928023

In [8]:
# SVR을 개선해봅니다.
from sklearn.svm import SVR

# DayName의 가변수를 추가
ct = ColumnTransformer([
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]), 
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName'])
])

reg_svm_2 = make_pipeline(ct, SVR(C=1000, kernel='rbf', gamma=0.1))
eval_model(reg_svm_2)

(127.89126981670202, 37.71620515264342)

In [9]:
# 성능 개선이 보입니다. 모델로 선정합니다.
prd = choose_model(reg_svm_2)
mean_absolute_error(s_kaggle_ans, prd)

31.97467882477439

In [10]:
# SVM + 대상 변수를 Log 변환
from sklearn.compose import TransformedTargetRegressor

ct = ColumnTransformer([
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]), 
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName'])
])

reg_svm_3 = make_pipeline(
    ct, 
    TransformedTargetRegressor(
        regressor = SVR(C=1, kernel='rbf', gamma=0.1),
        func=lambda x: np.log(x + 1),
        inverse_func = lambda x: np.exp(x) - 1
    )
)
eval_model(reg_svm_3)

(134.0816459387412, 46.3696367290434)

In [11]:
# SVM + PCA

from sklearn.decomposition import PCA
# DayName의 가변수를 추가
ct = ColumnTransformer([
    ('std', make_pipeline(StandardScaler(), PCA(n_components=7)), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]), 
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName'])
])

reg_svm_4 = make_pipeline(ct, SVR(C=100, kernel='rbf', gamma=0.1))
eval_model(reg_svm_4)

(130.14805081331278, 46.23430000312991)

In [12]:
# Gradient Boosting Model 입니다.
from sklearn.ensemble import GradientBoostingRegressor

X_gb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_gb = [i.strip() for i in X_gb.split(',')]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_gb)
])

reg_gb = make_pipeline(
    ct,
    GradientBoostingRegressor(
        loss = 'huber', # 손실함수: huber
        n_estimators = 75, # 트리의 수: 75
        max_depth = 7, # 트리의 최대 깊이: 7
        min_samples_leaf = 4, # 리프 노드(Leaf Node)의 최소 샘플 수: 4
        random_state = 123# 랜덤 시드: 123
    )
)
eval_model(reg_gb)

(117.32391481946111, 36.73714649971901)

In [13]:
# 성능 개선이 보입니다. 모델로 선정합니다.
prd = choose_model(reg_gb)
mean_absolute_error(s_kaggle_ans, prd)

38.86815607791259

In [14]:
# Gradient Boosting 모델 개선 
from sklearn.ensemble import GradientBoostingRegressor

X_gb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_gb = [i.strip() for i in X_gb.split(',')]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_gb)
])

reg_gb_1 = make_pipeline(
    ct,
    GradientBoostingRegressor(n_estimators=50, max_depth=6, random_state=123)
)
eval_model(reg_gb_1)

(108.87033477061964, 55.09825731799526)

In [15]:
# 성능 개선이 보입니다. 모델로 선정합니다.
prd = choose_model(reg_gb_1)
mean_absolute_error(s_kaggle_ans, prd)

43.08849195788369

In [16]:
# Gradient Boosting 모델 개선 - leaf wise 방법입니다.
from sklearn.ensemble import GradientBoostingRegressor

X_gb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_gb = [i.strip() for i in X_gb.split(',')]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_gb)
])

# max_depth: leaf-wise 
reg_gb_2 = make_pipeline(
    ct,
    GradientBoostingRegressor(n_estimators=100, max_depth=None, max_leaf_nodes=32, random_state=123)
)
eval_model(reg_gb_2)

(110.56255657727685, 39.98675230929889)

In [17]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

X_rf = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_rf = [i.strip() for i in X_rf.split(',')]
ct = ColumnTransformer([
    ('pt', 'passthrough', X_rf)
])

reg_rf = make_pipeline(
    ct,
    RandomForestRegressor(n_estimators=500, max_depth=13, random_state=123)
)
eval_model(reg_rf)

(108.17241670562379, 27.279992482983307)

In [18]:
# 성능 개선이 보입니다. 모델로 선정합니다.
prd = choose_model(reg_rf)
mean_absolute_error(s_kaggle_ans, prd)

33.55111041318366

In [19]:
from sklearn.linear_model import LinearRegression

# DayName의 가변수를 추가
ct = ColumnTransformer([
    ('std', StandardScaler(), ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]), 
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName'])
])

reg_lr = make_pipeline(ct, LinearRegression())
eval_model(reg_lr)

(164.05876778505638, 84.94987496121442)

In [20]:
# XGBoost 모델입니다.

import xgboost as xgb

X_xgb = "15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_xgb = [i.strip() for i in X_xgb.split(',')]

ct = ColumnTransformer([
    ('pt', 'passthrough', X_xgb)
])
reg_xgb = make_pipeline(
    ct,
    xgb.XGBRegressor(n_estimators=250, max_depth=6, random_state=123)
)
eval_model(reg_xgb)

(104.42331921413381, 27.079157434456736)

In [21]:
# 성능 개선이 보입니다. 모델로 선정합니다.
prd = choose_model(reg_xgb)
mean_absolute_error(s_kaggle_ans, prd)

46.65217423751469

In [22]:
# Voting을 해봅니다.
from sklearn.ensemble import VotingRegressor
reg_vt = VotingRegressor([
    #('svm_2', reg_svm_2),
    #('gb_1', reg_gb_1),
    #('gb_2', reg_gb_2),
    ('xgb', reg_xgb),
    ('rf_1', reg_rf)
])
eval_model(reg_vt)

(104.65663918982604, 25.921749323377654)

In [23]:
# 최종 모델로는 선정을 하지는 않을 것이지만 성능을 봅니다.
prd = choose_model(reg_vt)
mean_absolute_error(s_kaggle_ans, prd)

37.79926095113897